In [1]:
#Adapted and refered from "https://www.kaggle.com/code/itsmerasheed/named-entity-recognition-ner-with-tensorflow/notebook" 
#implemented By Eswar Veerapaneni

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from ast import literal_eval

In [3]:
data = pd.read_csv("ner_dataset.csv", encoding= 'unicode_escape')
data2 = pd.read_csv("ner.csv", encoding='unicode_escape')

In [4]:
tags = data.Tag.unique()
sentenses = data2['Sentence'].tolist()
getAllTags = data2['Tag'].tolist()

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentenses)
sequences = tokenizer.texts_to_sequences(sentenses)
wordsToId = tokenizer.word_index

In [6]:
idToWords = {}
for i, v in wordsToId.items():
    idToWords[v] = i

In [7]:
ConvertId = {}
ConvertTags = {}
i = 0
for tag in tags:
    ConvertTags[tag] = i
    ConvertId[i] = tag
    i = i + 1

In [8]:
getTags = []
for sentenses in getAllTags:
    getTags.append(literal_eval(sentenses))
allTags = []
for Tag in getTags:
    eachTag = []
    for tag in Tag:
        eachTag.append(ConvertTags[tag]) 
    reshapeTag = eachTag + ([ConvertTags['O']] * (200 - len(eachTag)))
    allTags.append(reshapeTag)
getTags

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-per',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'I-geo',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'B-org',
  'I-org',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'B-geo',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'B-geo',
  'O',
  'O',
  'O',
  '

In [9]:
allIds = pad_sequences(sequences, maxlen=200, padding='post')
allIds = np.asarray(allIds)
allTags = np.asarray(allTags)

In [10]:
X_train, X_rem, y_train, y_rem = train_test_split(allIds,allTags, train_size=0.70)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)


train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train = train.shuffle(255).batch(255)
validation = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
validation = validation.batch(255)
test = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test = test.batch(255)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(40000, 200),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=100, activation='softmax', return_sequences=True)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(tags), activation='softmax'))
])
model.summary()
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train,validation_data=validation,epochs=15)
model.evaluate(test)

In [ ]:
ShuffledS=  X_test[175] #get the random sentense
ShuffledS = ShuffledS.reshape((1, 200))
words = []
for word in ShuffledS[ShuffledS > 0].tolist():
    words.append(idToWords[word])
raw = ' '.join(words)
print(raw)  
getPredictions = model.predict(ShuffledS)
getPredictions = np.argmax(getPredictions[0], axis=1)
getPredictions = getPredictions.tolist()[ : len(words)] 
PredictTags = []
for ids in getPredictions:
    print(ConvertId[ids] + " ")
